In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/carlalopez/documents/Lie_Symmetry_Analysis_Agorithm/src')

In [3]:
from symmetries.analysis.lie_symmetry_analysis import point_symmetries
import sympy as sp
from sympy import Derivative as D

# One equation of Hall MHD 

In [4]:
x, y, z, t = sp.symbols('x y z t')

phi = sp.Function('phi')(x, y, z, t)
psi = sp.Function('psi')(x, y, z, t)
b = sp.Function('b')(x, y, z, t)
w = sp.Function('w')(x, y, z, t)

sigma, gamma = sp.symbols('sigma gamma')

list_indep = [x, y, z, t]
list_dep = [phi, psi, b, w]
list_cte = [sigma, gamma]

F =  D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) + sigma*(D(psi, x)*(D(psi, x, x, y) + D(psi, y, y, y))-
        D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))) + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) - gamma*(D(b,x,x) + D(b, y, y))

F_rules_array = {D(psi, x, x, y): 
        - (sigma*D(psi, x))**(-1)*(D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) 
        + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) 
        - gamma*(D(b,x,x) + D(b, y, y)))
        +(D(psi, x))**(-1)*D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))
        - D(psi, y, y, y)}

order = 3
F.expand()

-gamma*Derivative(b(x, y, z, t), (x, 2)) - gamma*Derivative(b(x, y, z, t), (y, 2)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (y, 3)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (x, 2), y) - sigma*Derivative(psi(x, y, z, t), (x, 3))*Derivative(psi(x, y, z, t), y) - sigma*Derivative(psi(x, y, z, t), y)*Derivative(psi(x, y, z, t), (y, 2), x) + Derivative(b(x, y, z, t), t) + Derivative(b(x, y, z, t), x)*Derivative(phi(x, y, z, t), y) - Derivative(b(x, y, z, t), y)*Derivative(phi(x, y, z, t), x) + Derivative(psi(x, y, z, t), x)*Derivative(w(x, y, z, t), y) - Derivative(psi(x, y, z, t), y)*Derivative(w(x, y, z, t), x)

# Heat Equation

In [3]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
independent_variables = [x, t]
dependent_variables = [u]
constants = [k]
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [4]:
model, system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

In [6]:
general_form = GeneralForm(model, system_of_equations)

In [7]:
general_form.print_determining_equations()

/Users/carlalopez/documents/Lie_Symmetry_Analysis_Agorithm/src/symmetries/objects/general_form.py:133: SymPyDeprecationWarning: 

non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-non-expr-in-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  row = sympy.Matrix([[i, sympy.Eq(get_symbolic_terms(
/Users/carlalopez/miniconda3/lib/python3.9/site-packages/sympy/matrices/common.py:493: SymPyDeprecationWarning: 

non-Expr objects in a Matrix is deprecated. Matrix represents
a mathematical matrix. To represent a container of non-numeric
entities, Use a list of lists, TableForm, NumPy array, or some
other data structure instead.

See https://docs.sympy.org/latest/explanation/ac

Matrix([
[1,                                 Eq(xi^(t)_u_u, 0)],
[2,                                 Eq(xi^(x)_u_u, 0)],
[3,                                   Eq(xi^(t)_u, 0)],
[4,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[5,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[6,                                   Eq(xi^(t)_x, 0)],
[7,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[8, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[9,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [8]:
system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

deleting xit u(x, t)
deleting xit x
found deleted variable in xit eq 4
found deleted variable in xit eq 4
found deleted variable in xit eq 6
found high order derivative of deleted variable in xit eq 0
deleting xix u(x, t)
found deleted variable in xix eq 3
found deleted variable in xix eq 3
found high order derivative of deleted variable in xix eq 1


In [29]:
system_of_equations.print_determining_equations()

Matrix([
[1,                                   Eq(xi^(t)_u, 0)],
[2,                                Eq(eta^(u)_u_u, 0)],
[3,                                   Eq(xi^(x)_u, 0)],
[4,                                   Eq(xi^(t)_x, 0)],
[5,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[6, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[7,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [12]:
system_of_equations.print_general_form()

already deleted: {'xit': [u(x, t), x], 'xix': [u(x, t)]}
general form:


Matrix([
[eta(x, t, u(x, t))**u],
[             xi(t)**t],
[          xi(x, t)**x]])

# Blasius Equation

In [18]:
x = sp.symbols('x')
y = sp.Function('y')(x)
independent_variables = [x]
dependent_variables = [y]
constants = []
F =  D(y,x,x,x) + y*D(y,x,x)
F_rules_array = {D(y,x,x,x):-y*D(y,x,x)}
order = 3

In [19]:
F.expand()

y(x)*Derivative(y(x), (x, 2)) + Derivative(y(x), (x, 3))

In [14]:
system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

In [21]:
system_of_equations.print_determining_equations()

Matrix([
[1,                                 Eq(xi^(t)_u_u, 0)],
[2,                                 Eq(xi^(x)_u_u, 0)],
[3,                                   Eq(xi^(t)_u, 0)],
[4,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[5,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[6,                                   Eq(xi^(t)_x, 0)],
[7,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[8, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[9,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [17]:
system_of_equations.general_form

{'xi': {'x': [x]}, 'eta': {'y': [x, y(x)]}}

# Wave Equation

In [16]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_var = list_indep + list_dep
list_cte = [k]
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [17]:
system_of_equations = point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

In [19]:
system_of_equations.print_determining_equations()

Matrix([
[1,                                 Eq(xi^(t)_u_u, 0)],
[2,                                 Eq(xi^(x)_u_u, 0)],
[3,                                   Eq(xi^(t)_u, 0)],
[4,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[5,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[6,                                   Eq(xi^(t)_x, 0)],
[7,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[8, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[9,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

# Schrödinger Equation

In [36]:
x, t, k, i = sp.symbols('x t k i')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_cte = [k, i]
F =  -D(u,x,x) +x**2*u-i*k*D(u,t)
F_rules_array = {D(u,x,x):x**2-i*k*D(u,t)}
order = 2
F.expand()

-i*k*Derivative(u(x, t), t) + x**2*u(x, t) - Derivative(u(x, t), (x, 2))

In [37]:
system_of_equations = point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

deleting xit u(x, t)
deleting xit x
found deleted variable in xit eq 3
found deleted variable in xit eq 3
found deleted variable in xit eq 6
found deleted variable in xit eq 6
found high order derivative of deleted variable in xit eq 0
deleting xix u(x, t)
found deleted variable in xix eq 4
found deleted variable in xix eq 4
found deleted variable in xix eq 7
found high order derivative of deleted variable in xix eq 1


In [38]:
system_of_equations.print_determining_equations()

Matrix([
[1,                                                                                           Eq(xi^(t)_u, 0)],
[2,                                                                                           Eq(xi^(x)_u, 0)],
[3,                                                                                        Eq(eta^(u)_u_u, 0)],
[4,                                                                                           Eq(xi^(t)_x, 0)],
[5,                                                        Eq(i*k*xi^(t)_t - 2*i*k*xi^(x)_x + x**2*xi^(t), 0)],
[6,                                           Eq(-2*eta^(u)_x_u + i*k*xi^(x)_t + x**2*xi^(x) + xi^(x)_x_x, 0)],
[7, Eq(eta^(u)*x**2 - eta^(u)_t*i*k - eta^(u)_u*x**2 - eta^(u)_x_x + 2*x**2*xi^(x)_x + 2*x*xi^(x)*u(x, t), 0)]])

# HNLS Equation

In [39]:
# Real
z, t, g1, g2, l, s = sp.symbols('z t gamma_1 gamma_2 lambda_0 sigma')
u = sp.Function('u')(z, t)
v = sp.Function('v')(z, t)
list_indep = [z, t]
list_dep = [u, v]
list_cte = [g1, g2, l, s]
F =  D(u,t) + D(v,z,z) + l*(v*u**2+v**3)+s*D(u,z,z,z)+\
    g1*D(u,z)*(u**2+v**2)+2*g2*u*(u*D(u,z)+v*D(v,z))
F_rules_array = {D(u,z,z,z): (-1/s)*(D(u,t) + D(v,z,z) + l*(v*u**2+v**3)+
    g1*D(u,z)*(u**2+v**2)+2*g2*u*(u*D(u,z)+v*D(v,z)))}
order = 3
F.expand()

gamma_1*u(z, t)**2*Derivative(u(z, t), z) + gamma_1*v(z, t)**2*Derivative(u(z, t), z) + 2*gamma_2*u(z, t)**2*Derivative(u(z, t), z) + 2*gamma_2*u(z, t)*v(z, t)*Derivative(v(z, t), z) + lambda_0*u(z, t)**2*v(z, t) + lambda_0*v(z, t)**3 + sigma*Derivative(u(z, t), (z, 3)) + Derivative(u(z, t), t) + Derivative(v(z, t), (z, 2))

In [40]:
system_of_equations = point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

In [41]:
system_of_equations.print_determining_equations()

Matrix([
[ 1,                                                                                                                                                                                                                                                                                                                                                                                                                                                     Eq(xi^(t)_u_u_u, 0)],
[ 2,                                                                                                                                                                                                                                                                                                                                                                                                                                                     Eq(xi^(t)_u_u_v, 0)],
[ 3,                                                             

In [34]:
system_of_equations.general_form

{'xi': {'z': [z, t], 't': [t]},
 'eta': {'u': [z, t, u(z, t)], 'v': [z, t, u(z, t), v(z, t)]}}

In [ ]:
system_of_equations.simplify_iteratively()

In [35]:
system_of_equations.print_determining_equations()

Matrix([
[ 1,                                                                                                                                                                                                                                                                                                                                                                                                                                Eq(xi^(t)_v, 0)],
[ 2,                                                                                                                                                                                                                                                                                                                                                                                                                                Eq(xi^(z)_v, 0)],
[ 3,                                                                                                               

In [ ]:
# Imaginaria
z, t, g1, g2, l, s = sp.symbols('z t gamma_1 gamma_2 lambda_0 sigma')
u = sp.Function('u')(z, t)
v = sp.Function('v')(z, t)
list_indep = [z, t]
list_dep = [u, v]
list_cte = [g1, g2, l, s]
F =-D(v,t) + D(u,z,z) + l*(v**2*u+u**3)-s*D(v,z,z,z)-\
    g1*D(v,z)*(u**2+v**2)-2*g2*v*(u*D(u,z)+v*D(v,z))
F_rules_array = {D(v,z,z,z): 1/s*(-D(v,t) + D(u,z,z) + l*(v**2*u+u**3)-\
    g1*D(v,z)*(u**2+v**2)-2*g2*v*(u*D(u,z)+v*D(v,z)))}
order = 3
F.expand()
point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)